In [9]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # set minimum logging to "info"

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import utils
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [10]:
# needed to deserialize dataset
features_description = utils.get_features_description()
# result is a single scene with a dictionary where the keys are as defined in features_description (e.g. "state/current/x", "state/current/y"...etc
# def parse_example_basic(value):
#     decoded_example = tf.io.parse_single_example(value, features_description)
#     return decoded_example

In [11]:


FILES = [
    "./data/training_tfexample.tfrecord-00000-of-01000",
]

dataset = tf.data.TFRecordDataset(FILES) # make dataset
dataset = dataset.map(parse_example_basic)


# take the first scene and inspect the shape of the data:
for example in dataset:
    print(example)
    for key, value in example.items():
        print(f"Key: {key}")
        print(f"Shape: {value.shape}")
        print(f"Type: {type(value)}")
        print("")
    break

NameError: name 'parse_example_basic' is not defined

In [ ]:
state_features = ["x", "y"]

# put past, current and future states of each feature together
def _parse_states(decoded_example):
#     past_states = [decoded_example['state/past/{}'.format(feature)] for feature in state_features if
#                    feature != "valid"]
#     past_states = tf.stack(past_states, -1)

#     current_states = [decoded_example['state/current/{}'.format(feature)] for feature in state_features if
#                       feature != "valid"]
#     current_states = tf.stack(current_states, -1)

#     future_states = [decoded_example['state/future/{}'.format(feature)] for feature in state_features if
#                      feature != "valid"]
#     future_states = tf.stack(future_states, -1)


#     # note this value will change later when we load data in batches instead of single scenes
#     axis = 1

#     # entire trajectory ground truth
#     states = tf.concat([past_states, current_states, future_states], axis)

#     past_is_valid = decoded_example['state/past/valid'] > 0
#     current_is_valid = decoded_example['state/current/valid'] > 0
#     future_is_valid = decoded_example['state/future/valid'] > 0
#     states_is_valid = tf.concat([past_is_valid, current_is_valid, future_is_valid], axis)
    
    cyclist_present = decoded_example['state/type'] == 3
    cyclist_present = tf.reduce_any(cyclist_present)
#     print(cyclist_present)
    # If a sample was not seen at all in the past, we declare the sample as invalid.
#     sample_is_valid = tf.reduce_any(tf.concat([past_is_valid, current_is_valid], axis), axis)

    return cyclist_present

def parse_examples(examples):
    decoded_examples = tf.io.parse_single_example(examples, features_description)
    cyclist_present = _parse_states(decoded_examples)
    decoded_examples['cyclist_present'] = cyclist_present

    return decoded_examples


dataset = tf.data.TFRecordDataset(FILES)
dataset = dataset.map(parse_examples)

# filter out cyclist data
cyclist_dataset = dataset.filter(lambda x: x['cyclist_present'] == True)




In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value is tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _float_feature(value):
  """Returns a floast_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array

In [ ]:
filename="test.tfrecords"
writer = tf.io.TFRecordWriter(filename) #create a writer that'll store our data to disk

count = 0
for sample in cyclist_dataset:
    data = {
            
            "scenario/id": _bytes_feature(sample['scenario/id']),
        
            'roadgraph_samples/dir': _float_feature(sample['roadgraph_samples/dir'].numpy().ravel().tolist()),
            'roadgraph_samples/id': _int64_feature(sample['roadgraph_samples/id'].numpy().ravel().tolist()),
            "roadgraph_samples/type": _int64_feature(sample['roadgraph_samples/type'].numpy().ravel().tolist()),
            "roadgraph_samples/valid": _int64_feature(sample['roadgraph_samples/valid'].numpy().ravel().tolist()),
            "roadgraph_samples/xyz": _float_feature(sample['roadgraph_samples/xyz'].numpy().ravel().tolist()),
            
            'state/id': _float_feature(sample['state/id']),
            'state/type': _float_feature(sample['state/type']),
            'state/is_sdc': _int64_feature(sample['state/is_sdc']),
            'state/tracks_to_predict': _int64_feature(sample['state/tracks_to_predict']),
        
            'state/current/bbox_yaw' : _float_feature(sample['state/current/bbox_yaw']),
            'state/current/height' : _float_feature(sample['state/current/height']),
            'state/current/length' : _float_feature(sample['state/current/length']),
            'state/current/timestamp_micros': _int64_feature(sample['state/current/timestamp_micros'].numpy().ravel().tolist()),
            'state/current/valid': _int64_feature(sample['state/current/valid'].numpy().ravel().tolist()),
            'state/current/vel_yaw' : _float_feature(sample['state/current/vel_yaw']),
            'state/current/velocity_x' : _float_feature(sample['state/current/velocity_x']),
            'state/current/velocity_y' : _float_feature(sample['state/current/velocity_y']),
            'state/current/speed' : _float_feature(sample['state/current/speed']),
            'state/current/width' : _float_feature(sample['state/current/width']),
            'state/current/x' : _float_feature(sample['state/current/x']),
            'state/current/y' : _float_feature(sample['state/current/y']),
            'state/current/z' : _float_feature(sample['state/current/z']),
        
            'state/future/bbox_yaw' : _float_feature(sample['state/future/bbox_yaw'].numpy().ravel().tolist()),
            'state/future/height' : _float_feature(sample['state/future/height'].numpy().ravel().tolist()),
            'state/future/length' : _float_feature(sample['state/future/length'].numpy().ravel().tolist()),
            'state/future/timestamp_micros': _int64_feature(sample['state/future/timestamp_micros'].numpy().ravel().tolist()),
            'state/future/valid': _int64_feature(sample['state/future/valid'].numpy().ravel().tolist()),
            'state/future/vel_yaw' : _float_feature(sample['state/future/vel_yaw'].numpy().ravel().tolist()),
            'state/future/velocity_x' : _float_feature(sample['state/future/velocity_x'].numpy().ravel().tolist()),
            'state/future/velocity_y' : _float_feature(sample['state/future/velocity_y'].numpy().ravel().tolist()),
            'state/future/speed' : _float_feature(sample['state/future/speed'].numpy().ravel().tolist()),
            'state/future/width' : _float_feature(sample['state/future/width'].numpy().ravel().tolist()),
            'state/future/x' : _float_feature(sample['state/future/x'].numpy().ravel().tolist()),
            'state/future/y' : _float_feature(sample['state/future/y'].numpy().ravel().tolist()),
            'state/future/z' : _float_feature(sample['state/future/z'].numpy().ravel().tolist()),
        
            'state/past/bbox_yaw' : _float_feature(sample['state/past/bbox_yaw'].numpy().ravel().tolist()),
            'state/past/height' : _float_feature(sample['state/past/height'].numpy().ravel().tolist()),
            'state/past/length' : _float_feature(sample['state/past/length'].numpy().ravel().tolist()),
            'state/past/timestamp_micros': _int64_feature(sample['state/past/timestamp_micros'].numpy().ravel().tolist()),
            'state/past/valid': _int64_feature(sample['state/past/valid'].numpy().ravel().tolist()),
            'state/past/vel_yaw' : _float_feature(sample['state/past/vel_yaw'].numpy().ravel().tolist()),
            'state/past/velocity_x' : _float_feature(sample['state/past/velocity_x'].numpy().ravel().tolist()),
            'state/past/velocity_y' : _float_feature(sample['state/past/velocity_y'].numpy().ravel().tolist()),
            'state/past/speed' : _float_feature(sample['state/past/speed'].numpy().ravel().tolist()),
            'state/past/width' : _float_feature(sample['state/past/width'].numpy().ravel().tolist()),
            'state/past/x' : _float_feature(sample['state/past/x'].numpy().ravel().tolist()),
            'state/past/y' : _float_feature(sample['state/past/y'].numpy().ravel().tolist()),
            'state/past/z' : _float_feature(sample['state/past/z'].numpy().ravel().tolist()),
        
            'traffic_light_state/current/state': _int64_feature(sample['traffic_light_state/current/state'].numpy().ravel().tolist()),
            'traffic_light_state/current/valid': _int64_feature(sample['traffic_light_state/current/valid'].numpy().ravel().tolist()),
            'traffic_light_state/current/id': _int64_feature(sample['traffic_light_state/current/id'].numpy().ravel().tolist()),
            'traffic_light_state/current/x': _float_feature(sample['traffic_light_state/current/x'].numpy().ravel().tolist()),
            'traffic_light_state/current/y': _float_feature(sample['traffic_light_state/current/y'].numpy().ravel().tolist()),
            'traffic_light_state/current/z': _float_feature(sample['traffic_light_state/current/z'].numpy().ravel().tolist()),
            
            'traffic_light_state/past/state': _int64_feature(sample['traffic_light_state/past/state'].numpy().ravel().tolist()),
            'traffic_light_state/past/valid': _int64_feature(sample['traffic_light_state/past/valid'].numpy().ravel().tolist()),
            'traffic_light_state/past/x': _float_feature(sample['traffic_light_state/past/x'].numpy().ravel().tolist()),
            'traffic_light_state/past/y': _float_feature(sample['traffic_light_state/past/y'].numpy().ravel().tolist()),
            'traffic_light_state/past/z': _float_feature(sample['traffic_light_state/past/z'].numpy().ravel().tolist()),
        
             
    }
    
    out = tf.train.Example(features=tf.train.Features(feature=data))
    writer.write(out.SerializeToString())
    count += 1

writer.close()
print(f"Wrote {count} elements to TFRecord")




In [ ]:
FILES = [
    "test.tfrecords",
]

dataset = tf.data.TFRecordDataset(FILES) # make dataset
dataset = dataset.map(parse_example_basic)

In [ ]:
for example in dataset:
    print(example)
    for key, value in example.items():
        print(f"Key: {key}")
        print(f"Shape: {value.shape}")
        print(f"Type: {type(value)}")
        print("")
    break

In [16]:
# put past, current and future states of each feature together
def _parse_states(decoded_example):    
    cyclist_present = decoded_example['state/type'] == 3
    cyclist_present = tf.reduce_any(cyclist_present)

    return cyclist_present

def parse_examples(examples):
    decoded_examples = tf.io.parse_single_example(examples, features_description)
    cyclist_present = _parse_states(decoded_examples)
    decoded_examples['cyclist_present'] = cyclist_present
    return decoded_examples

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value is tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _float_feature(value):
  """Returns a floast_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array

In [22]:
for file in os.listdir('data'):
    FILES = [
    "./data/" + file,
    ]

    dataset = tf.data.TFRecordDataset(FILES) # make dataset
    dataset = dataset.map(parse_examples)
    original_size = len(list(dataset))
    cyclist_dataset = dataset.filter(lambda x: x['cyclist_present'] == True)
    
    writer = tf.io.TFRecordWriter("./data_cleaned/"+ file) #create a writer that'll store our data to disk

    count = 0
    for sample in cyclist_dataset:
        data = {

                "scenario/id": _bytes_feature(sample['scenario/id']),

                'roadgraph_samples/dir': _float_feature(sample['roadgraph_samples/dir'].numpy().ravel().tolist()),
                'roadgraph_samples/id': _int64_feature(sample['roadgraph_samples/id'].numpy().ravel().tolist()),
                "roadgraph_samples/type": _int64_feature(sample['roadgraph_samples/type'].numpy().ravel().tolist()),
                "roadgraph_samples/valid": _int64_feature(sample['roadgraph_samples/valid'].numpy().ravel().tolist()),
                "roadgraph_samples/xyz": _float_feature(sample['roadgraph_samples/xyz'].numpy().ravel().tolist()),

                'state/id': _float_feature(sample['state/id']),
                'state/type': _float_feature(sample['state/type']),
                'state/is_sdc': _int64_feature(sample['state/is_sdc']),
                'state/tracks_to_predict': _int64_feature(sample['state/tracks_to_predict']),

                'state/current/bbox_yaw' : _float_feature(sample['state/current/bbox_yaw']),
                'state/current/height' : _float_feature(sample['state/current/height']),
                'state/current/length' : _float_feature(sample['state/current/length']),
                'state/current/timestamp_micros': _int64_feature(sample['state/current/timestamp_micros'].numpy().ravel().tolist()),
                'state/current/valid': _int64_feature(sample['state/current/valid'].numpy().ravel().tolist()),
                'state/current/vel_yaw' : _float_feature(sample['state/current/vel_yaw']),
                'state/current/velocity_x' : _float_feature(sample['state/current/velocity_x']),
                'state/current/velocity_y' : _float_feature(sample['state/current/velocity_y']),
                'state/current/speed' : _float_feature(sample['state/current/speed']),
                'state/current/width' : _float_feature(sample['state/current/width']),
                'state/current/x' : _float_feature(sample['state/current/x']),
                'state/current/y' : _float_feature(sample['state/current/y']),
                'state/current/z' : _float_feature(sample['state/current/z']),

                'state/future/bbox_yaw' : _float_feature(sample['state/future/bbox_yaw'].numpy().ravel().tolist()),
                'state/future/height' : _float_feature(sample['state/future/height'].numpy().ravel().tolist()),
                'state/future/length' : _float_feature(sample['state/future/length'].numpy().ravel().tolist()),
                'state/future/timestamp_micros': _int64_feature(sample['state/future/timestamp_micros'].numpy().ravel().tolist()),
                'state/future/valid': _int64_feature(sample['state/future/valid'].numpy().ravel().tolist()),
                'state/future/vel_yaw' : _float_feature(sample['state/future/vel_yaw'].numpy().ravel().tolist()),
                'state/future/velocity_x' : _float_feature(sample['state/future/velocity_x'].numpy().ravel().tolist()),
                'state/future/velocity_y' : _float_feature(sample['state/future/velocity_y'].numpy().ravel().tolist()),
                'state/future/speed' : _float_feature(sample['state/future/speed'].numpy().ravel().tolist()),
                'state/future/width' : _float_feature(sample['state/future/width'].numpy().ravel().tolist()),
                'state/future/x' : _float_feature(sample['state/future/x'].numpy().ravel().tolist()),
                'state/future/y' : _float_feature(sample['state/future/y'].numpy().ravel().tolist()),
                'state/future/z' : _float_feature(sample['state/future/z'].numpy().ravel().tolist()),

                'state/past/bbox_yaw' : _float_feature(sample['state/past/bbox_yaw'].numpy().ravel().tolist()),
                'state/past/height' : _float_feature(sample['state/past/height'].numpy().ravel().tolist()),
                'state/past/length' : _float_feature(sample['state/past/length'].numpy().ravel().tolist()),
                'state/past/timestamp_micros': _int64_feature(sample['state/past/timestamp_micros'].numpy().ravel().tolist()),
                'state/past/valid': _int64_feature(sample['state/past/valid'].numpy().ravel().tolist()),
                'state/past/vel_yaw' : _float_feature(sample['state/past/vel_yaw'].numpy().ravel().tolist()),
                'state/past/velocity_x' : _float_feature(sample['state/past/velocity_x'].numpy().ravel().tolist()),
                'state/past/velocity_y' : _float_feature(sample['state/past/velocity_y'].numpy().ravel().tolist()),
                'state/past/speed' : _float_feature(sample['state/past/speed'].numpy().ravel().tolist()),
                'state/past/width' : _float_feature(sample['state/past/width'].numpy().ravel().tolist()),
                'state/past/x' : _float_feature(sample['state/past/x'].numpy().ravel().tolist()),
                'state/past/y' : _float_feature(sample['state/past/y'].numpy().ravel().tolist()),
                'state/past/z' : _float_feature(sample['state/past/z'].numpy().ravel().tolist()),

                'traffic_light_state/current/state': _int64_feature(sample['traffic_light_state/current/state'].numpy().ravel().tolist()),
                'traffic_light_state/current/valid': _int64_feature(sample['traffic_light_state/current/valid'].numpy().ravel().tolist()),
                'traffic_light_state/current/id': _int64_feature(sample['traffic_light_state/current/id'].numpy().ravel().tolist()),
                'traffic_light_state/current/x': _float_feature(sample['traffic_light_state/current/x'].numpy().ravel().tolist()),
                'traffic_light_state/current/y': _float_feature(sample['traffic_light_state/current/y'].numpy().ravel().tolist()),
                'traffic_light_state/current/z': _float_feature(sample['traffic_light_state/current/z'].numpy().ravel().tolist()),

                'traffic_light_state/past/state': _int64_feature(sample['traffic_light_state/past/state'].numpy().ravel().tolist()),
                'traffic_light_state/past/valid': _int64_feature(sample['traffic_light_state/past/valid'].numpy().ravel().tolist()),
                'traffic_light_state/past/x': _float_feature(sample['traffic_light_state/past/x'].numpy().ravel().tolist()),
                'traffic_light_state/past/y': _float_feature(sample['traffic_light_state/past/y'].numpy().ravel().tolist()),
                'traffic_light_state/past/z': _float_feature(sample['traffic_light_state/past/z'].numpy().ravel().tolist()),


        }

        out = tf.train.Example(features=tf.train.Features(feature=data))
        writer.write(out.SerializeToString())
        count += 1

    writer.close()
    print("Wrote "+ str(count) + " out of " + str(original_size) + " elements to TFRecord " + str(count/original_size))




Wrote 114 out of 478 elements to TFRecord 0.2384937238493724
Wrote 122 out of 518 elements to TFRecord 0.23552123552123552
Wrote 140 out of 514 elements to TFRecord 0.2723735408560311
Wrote 124 out of 479 elements to TFRecord 0.2588726513569937
Wrote 112 out of 435 elements to TFRecord 0.2574712643678161
Wrote 126 out of 512 elements to TFRecord 0.24609375
Wrote 134 out of 501 elements to TFRecord 0.26746506986027946
Wrote 116 out of 487 elements to TFRecord 0.23819301848049282
Wrote 124 out of 487 elements to TFRecord 0.2546201232032854
Wrote 121 out of 531 elements to TFRecord 0.2278719397363465
Wrote 116 out of 520 elements to TFRecord 0.2230769230769231
Wrote 136 out of 493 elements to TFRecord 0.27586206896551724
Wrote 120 out of 492 elements to TFRecord 0.24390243902439024
Wrote 123 out of 450 elements to TFRecord 0.2733333333333333
Wrote 137 out of 503 elements to TFRecord 0.27236580516898606
Wrote 116 out of 446 elements to TFRecord 0.2600896860986547
Wrote 130 out of 510 eleme

In [32]:
for example in cyclist_dataset:
    for key, value in example.items():
        if key == 'state/tracks_to_predict':
            print(value)
        if key == 'state/type':
            print(value)

tf.Tensor(
[ 1  1  1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1], shape=(128,), dtype=int64)
tf.Tensor(
[ 1.  1.  1.  1.  1.  2.  3.  1.  1.  1.  1.  1.  1.  1.  2.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  2.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  2. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1.], shape=(128,), dtype=float32)
tf.Tenso

tf.Tensor(
[ 1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1], shape=(128,), dtype=int64)
tf.Tensor(
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  2.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  2.  2.  1.  1.  1.  1.  1.  2.  1.  1.
  1.  1.  1.  2.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  2.  3.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  2.  2.  2. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1.], shape=(128,), dtype=float32)
tf.Tenso

tf.Tensor(
[ 1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1], shape=(128,), dtype=int64)
tf.Tensor(
[ 1.  1.  1.  3.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  2.  2.  2.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1.], shape=(128,), dtype=float32)
tf.Tenso

tf.Tensor(
[ 1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1], shape=(128,), dtype=int64)
tf.Tensor(
[ 1.  2.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  3. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1.], shape=(128,), dtype=float32)
tf.Tenso